In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df1=pd.read_csv('movies_metadata.csv')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df2=pd.read_csv('credits.csv')

In [4]:
#remove invalid id rows
df1=df1[df1.id !='1997-08-20']
df1=df1[df1.id !='2012-09-29']
df1=df1[df1.id !='2014-01-01']

In [5]:
#converge to same type
df1=df1.astype({"id":int})

In [6]:
df=df1.merge(df2,on='id')

In [7]:
#Weighted Average
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [8]:
#C is the mean vote of all movies.
C= df['vote_average'].mean()

In [9]:
#m is the minimum votes required. We select the top 80 percent.
m= df['vote_count'].quantile(0.8)

In [10]:
R = df['vote_average']
v = df['vote_count']

In [11]:
df['weighted_average'] = (R*v + C*m)/(v+m)

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[['weighted_average']])
#weighted_df = pd.DataFrame(scaled, columns=['popularity', 'weighted_average'])

In [13]:
df['weighted_average']=scaled


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tfidf = TfidfVectorizer(stop_words='english')

In [16]:
type(df['cast'][0])

str

In [17]:
#Take string input and evaluate strings with python values from unknown sources without parsing the values.
#convert string into list
from ast import literal_eval

attributes = ['cast', 'crew', 'genres']
for attribute in attributes:
    df[attribute] = df[attribute].apply(literal_eval)

In [18]:
type(df['cast'][0])

list

In [19]:
#Get director from crews
def director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return ' '

In [20]:
df['director'] = df['crew'].apply(director)

In [21]:
#Get necessary info from attributes in the forms of dictionaries.
def get_names(x):
    if isinstance(x, list):#check if input is a list.
        names=[]
        for i in x:
            names.append(i['name'])
        if len(names) > 5:
            names = names[:5]
        return names
    return []

In [22]:
attributes = ['cast','crew','genres']
for attribute in attributes:
    df[attribute] = df[attribute].apply(get_names)

In [23]:
df[['title', 'cast', 'director', 'crew', 'genres']].head(5)

,title,cast,director,crew,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,"[John Lasseter, Joss Whedon, Andrew Stanton, J...","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston,"[Larry J. Franco, Jonathan Hensleigh, James Ho...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch,"[Howard Deutch, Mark Steven Johnson, Mark Stev...","[Romance, Comedy]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker,"[Forest Whitaker, Ronald Bass, Ronald Bass, Ez...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer,"[Alan Silvestri, Elliot Davis, Nancy Meyers, N...",[Comedy]


In [24]:
# Convert all strings to lower case and get rid of spaces.
def dat_clean(x):
    if isinstance(x, list):
        clean=[]
        for i in x:
            clean.append(i.replace(" ", "").lower())
        return clean
    else:
        #directors in strings format
        if isinstance(x, str):
            return x.replace(" ", "").lower()
        else:
            return ''

In [25]:
df['overview'] = df['overview'].fillna('')

In [26]:
attributes = ['cast', 'director','crew','genres','overview']

for attribute in attributes:
    df[attribute] = df[attribute].apply(dat_clean)

In [27]:
df[['original_title', 'cast', 'director', 'crew', 'genres']].head(5)

,original_title,cast,director,crew,genres
0,Toy Story,"[tomhanks, timallen, donrickles, jimvarney, wa...",johnlasseter,"[johnlasseter, josswhedon, andrewstanton, joel...","[animation, comedy, family]"
1,Jumanji,"[robinwilliams, jonathanhyde, kirstendunst, br...",joejohnston,"[larryj.franco, jonathanhensleigh, jameshorner...","[adventure, fantasy, family]"
2,Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret, sophi...",howarddeutch,"[howarddeutch, markstevenjohnson, markstevenjo...","[romance, comedy]"
3,Waiting to Exhale,"[whitneyhouston, angelabassett, lorettadevine,...",forestwhitaker,"[forestwhitaker, ronaldbass, ronaldbass, ezras...","[comedy, drama, romance]"
4,Father of the Bride Part II,"[stevemartin, dianekeaton, martinshort, kimber...",charlesshyer,"[alansilvestri, elliotdavis, nancymeyers, nanc...",[comedy]


In [28]:
#Combine all of the strings together
def combine_together(x):
    return ' '.join(x['cast']) + ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['genres'])+' '+' '.join(x['overview'])
df['sum'] = df.apply(combine_together, axis=1)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_vect = count.fit_transform(df['sum'])

In [30]:
#Calculate the similarties between two matrices.
from sklearn.metrics.pairwise import cosine_similarity

cossim_score = cosine_similarity(count_vect, count_vect)

In [31]:
# We put higher weight on similiarity and then consider the movies with high ratings.
# We only wish to output the first few movies with the highest overall scores.
def similar(title, similarity_weight,n):
    new_df = df.reset_index()
    movie_index = new_df[new_df['original_title'] == title].index
    sim_val = cossim_score[movie_index].T
    
    sim_df = pd.DataFrame(sim_val, columns=['sim_value'])
    final_df = pd.concat([new_df, sim_df], axis=1)
    final_df['overall_score'] = final_df['weighted_average']*(1-similarity_weight) + final_df['sim_value']*similarity_weight
    
    final_df= final_df.sort_values(by='overall_score', ascending=False).head(n)
    final_df.set_index('title', inplace=True)
    return final_df[['weighted_average', 'sim_value', 'overall_score']]

In [32]:
similar('Toy Story',0.8,10)

,weighted_average,sim_value,overall_score
title,,,
Toy Story,0.812252,1.000000,0.962450
Toy Story 2,0.747948,0.750000,0.749590
Toy Story of Terror!,0.705917,0.750000,0.741183
Partysaurus Rex,0.658591,0.707107,0.697404
Toy Story That Time Forgot,0.639794,0.668153,0.662481
Toy Story 3,0.795980,0.625000,0.659196
Creature Comforts,0.581146,0.612372,0.606127
Lorenzo,0.523769,0.612372,0.594652
In a Heartbeat,0.801847,0.530330,0.584633
